In [7]:
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient
import numpy as np
import mne
from pipeline.utils import make_mne_epochs, make_epochs_tfr

In [ ]:
sbj, session = 9, 3

fname = "sub-{0:>02d}_ses-{1:.0f}_behavior+ecephys.nwb".format(sbj, session)

cloud_fpath = "sub-{0:>02d}/sub-{0:>02d}_ses-{1:.0f}_behavior+ecephys.nwb".format(sbj, session)
print("Retrieving cloud file...")
with DandiAPIClient() as client:

    print("Starting a Dandi streaming client...")

    asset = client.get_dandiset("000055", "draft").get_asset_by_path(cloud_fpath)
    s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)

io = NWBHDF5IO(s3_path, mode='r', load_namespaces=True, driver='ros3')
nwb = io.read()

In [ ]:
neural_data = nwb.acquisition['ElectricalSeries'].data

print("Neural data shape: ", neural_data.shape)
n_times, n_chan = neural_data.shape
SF = nwb.acquisition['ElectricalSeries'].rate
print("Sampling rate: ", SF)

info = mne.create_info(n_chan, sfreq=SF)


mne_data_chunk = mne.io.RawArray(neural_data[:].reshape(n_chan, -1), info)

Neural data shape:  (43200000, 126)
Sampling rate:  500.0


In [10]:
trials = np.array(nwb.intervals['reaches']['start_time'].data) * SF
trials = trials[(trials > SF * 2) & (trials < SF * 2)]
print("N trials: ", len(trials))

N trials:  0


In [14]:
epochs = make_mne_epochs(mne_data_chunk, trials, -0.5, 1.5)
epochs.info

array([24077.1  , 24262.754, 34223.493, 34267.593, 39710.516, 41278.43 ,
       62912.246, 62953.313, 63344.012, 63620.388, 63626.255, 72038.407,
       72606.914, 73310.556, 75959.187])

In [ ]:
epochs_tfr = make_epochs_tfr(epochs)
epochs_tfr.info

In [ ]:
avgpower = epochs_tfr.average()
avgpower.plot(title="Average power")